In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from progress.bar import Bar, ChargingBar
import os, time, random
from tqdm import tqdm

In [2]:
from bs4 import BeautifulSoup
import requests

LOGIN_URL = "https://app.flowalgo.com/users/login"


def get_authenticity_token(html):
    soup = BeautifulSoup(html, "html.parser")
    token = soup.find('input', attrs={'name': 'login_attempt_id'})
    if not token:
        print('could not find `authenticity_token` on login form')
    return token.get('value').strip()

email = "********" 
password = "******" 

payload = {
    'amember_login': email,
    'amember_pass': password,
    'utf8': '&#x2713;',
}

session = requests.Session()
session.headers = {'User-Agent': ('Mozilla / 5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit / 537.36 (KHTML, como Gecko) Chrome / 39.0.2171.95 Safari / 537.36')}
response = session.get(LOGIN_URL)

token = get_authenticity_token(response.text)
payload.update({
    'login_attempt_id': token,
})

print(f"attempting to log in as {email}")
p = session.post(LOGIN_URL, data=payload) 

attempting to log in as alvar0291


In [3]:
def sacar_soup(session,url_page):
    with session as session:
        post = p
        r = session.get(url_page)
        soup = BeautifulSoup(r.text, "html.parser")
        return soup

In [4]:
def crear_tabla(soup):
    #date
    date = [item.get_text(strip=True) for item in soup.select("div.date")]
    date = date[1:-1]
    #time
    time = [item.get_text(strip=True) for item in soup.select("div.time")]
    time = time[1:-1]
    #ticker
    ticker = [item.get_text(strip=True) for item in soup.select("div.ticker")]
    ticker = ticker[1:-1]
    #expiry
    expiry = [item.get_text(strip=True) for item in soup.select("div.expiry")]
    expiry = expiry[1:]
    for i in range(0,len(expiry)):
        expiry[i] = expiry[i][6:]
    #strike
    strike = [item.get_text(strip=True) for item in soup.select("div.strike")]
    strike = strike[1:]
    for i in range(0,len(strike)):
        strike[i] = strike[i][6:]
    #C/P
    c_p = [item.get_text(strip=True) for item in soup.select("div.contract-type")]
    c_p = c_p[1:]
    for i in range(0,len(c_p)):
        c_p[i] = c_p[i][3:]
    #spot
    spot = [item.get_text(strip=True) for item in soup.select("div.ref")]
    spot = spot[1:-1]
    for i in range(0,len(spot)):
        spot[i] = spot[i][10:]
    #details
    details = [item.get_text(strip=True) for item in soup.select("div.details")]
    volume_operation = []
    price_operation = []
    details = details[1:]
    for i in range(0,len(details)):
        details[i] = details[i][21:]
    for i in range(0,len(details)):
        _ = details[i].split('@')
        voloperation = _[0].strip()
        priceoperation = _[1].strip()
        volume_operation.append(voloperation)
        price_operation.append(priceoperation)
    for i in range(0,len(volume_operation)):
        volume_operation[i] = volume_operation[i].replace(',', '')
    #Type
    tipo = [item.get_text(strip=True) for item in soup.select("div.type")]
    tipo = tipo[1:]
    #vol
    vol = [item.get_text(strip=True) for item in soup.select("div.volume")]
    vol = vol[1:]
    for i in range(0,len(vol)):
        vol[i] = vol[i].replace(',', '')
    #OI
    open_interest = [item.get_text(strip=True) for item in soup.select("div.open-interest")]
    open_interest = open_interest[1:]
    for i in range(0,len(open_interest)):
        open_interest[i] = open_interest[i].replace(',', '')

    
    data_tuples = list(zip(date,time,ticker,expiry,strike,c_p,spot,volume_operation,price_operation,tipo,vol,open_interest))
    BBDD = pd.DataFrame(data_tuples,
                   columns = ['date', 
                              'time', 
                              'ticker', 
                              'expiry', 
                              'strike', 
                              'c_p', 'spot', 
                              'volume_operation', 
                              'price_operation', 
                              'tipo', 
                              'vol', 
                              'open_interest'])
    BBDD[['strike', 
          'spot', 
          'volume_operation', 
          'price_operation', 
          'vol', 
          'open_interest']] = BBDD[['strike', 
                                    'spot', 
                                    'volume_operation', 
                                    'price_operation', 
                                    'vol', 
                                    'open_interest']].apply(pd.to_numeric)
    BBDD['money'] = BBDD['volume_operation'] * BBDD['price_operation'] 
    BBDD['date'] = BBDD['date'] + ' ' + BBDD['time']
    BBDD = BBDD.drop(['time'], axis = 1)
    BBDD['date'] = pd.to_datetime(BBDD['date'], format = '%m/%d/%y %I:%M %p')
    BBDD['date'] = pd.to_datetime(BBDD['date'], format = '%d/%m/%y %H:%M')
    BBDD['expiry'] = pd.to_datetime(BBDD['expiry'], format = '%m/%d/%y')
    BBDD['expiry'] = pd.to_datetime(BBDD['expiry'], format = '%d/%m/%y')
    BBDD.set_index('date', inplace = True)
    
    
    return BBDD


In [5]:
url_page = 'https://app.flowalgo.com/historical-flow/?tickers=&start=&end=&minsize=&show=500'

In [6]:
soup = sacar_soup(session,url_page)
BBDD = crear_tabla(soup)

In [7]:
tickers = list(BBDD['ticker'])
tickers = np.unique(tickers)

In [8]:
tickers

array(['AAPL', 'ABBV', 'ABNB', 'AFRM', 'AGNC', 'AHT', 'ALGN', 'AMC',
       'AMD', 'AMZN', 'APA', 'API', 'AQUA', 'ARKK', 'ATOS', 'ATVI',
       'AVGO', 'AYX', 'BABA', 'BAC', 'BB', 'BBIG', 'BGFV', 'BHF', 'BMY',
       'BP', 'BTU', 'BX', 'C', 'CAR', 'CCJ', 'CCL', 'CDE', 'CHWY', 'CKPT',
       'CLF', 'CLOV', 'CMCSA', 'CNK', 'CNST', 'COG', 'CRBP', 'CRIS',
       'CRSR', 'CSCO', 'CVM', 'CWH', 'DAL', 'DBX', 'DDD', 'DIS', 'DOCU',
       'DPW', 'DRI', 'EBAY', 'EDU', 'EEM', 'ELY', 'ENB', 'ET', 'EVFM',
       'EWP', 'EWZ', 'F', 'FAII', 'FB', 'FCX', 'FEYE', 'FOA', 'FSLR',
       'FSR', 'FUBO', 'GE', 'GEO', 'GGAL', 'GLD', 'GME', 'GOGO', 'GTT',
       'HL', 'IAA', 'IDEX', 'INTC', 'IRM', 'IVR', 'IWN', 'JNPR', 'KBH',
       'KHC', 'KLR', 'KO', 'KODK', 'KSS', 'LEV', 'LH', 'LI', 'LOTZ',
       'LQD', 'MA', 'MAC', 'MCD', 'MGI', 'MGNI', 'MNMD', 'MOMO', 'MRK',
       'MSFT', 'MT', 'MU', 'MUDS', 'MUX', 'MX', 'NCLH', 'NEE', 'NFLX',
       'NGL', 'NIO', 'NOK', 'NVDA', 'OCGN', 'ORCL', 'OTLY', 'OXY', 'PAA',
  

In [48]:
sdate = date(2017,6,2)

In [49]:
edate = date(2018,12,31)

In [50]:
dias = pd.date_range(sdate,edate-timedelta(days=1),freq='d')

In [51]:
url1 = 'https://app.flowalgo.com/historical-flow/?tickers='
url2 = '&start='
url3 = '%2F'
url4 = '&end='
url5 = '&minsize=&show=500'

In [52]:
dia = list(dias.strftime('%d'))

In [53]:
mes = list(dias.strftime('%m'))

In [54]:
año = list(dias.strftime('%Y'))

In [69]:
lista_url = []

In [17]:
len(tickers)

185

In [16]:
for i in range(len(dia)+1):
    if i<len(dia)-1:
        diario = dia[i]
        mensual = mes[i]
        anual = año[i]
        dia_despues = dia[i+1]
        mes_despues = mes[i+1]
        año_despues = año[i+1]
        for t in range(len(tickers)):
            ticker = tickers[t]
            url = (url1+str(ticker)+url2+str(mensual)+url3+str(diario)+url3+str(anual)+url4+str(mes_despues)+url3+str(dia_despues)+url3+str(año_despues)+url5)
            lista_url.append(url)

In [70]:
for i in range(len(dia)+1):
    if i<len(dia)-1:
        diario = dia[i]
        mensual = mes[i]
        anual = año[i]
        dia_despues = dia[i+1]
        mes_despues = mes[i+1]
        año_despues = año[i+1]
        ticker = 'BAC'
        url = (url1+str(ticker)+url2+str(mensual)+url3+str(diario)+url3+str(anual)+url4+str(mes_despues)+url3+str(dia_despues)+url3+str(año_despues)+url5)
        lista_url.append(url)

In [72]:
lo_consegui = pd.DataFrame()

In [73]:
for i in tqdm(range(len(lista_url))):
    url_page = lista_url[i]
    soup = sacar_soup(session,url_page)
    BBDD = crear_tabla(soup)
    unir = [BBDD, lo_consegui]
    lo_consegui = pd.concat(unir)


100%|██████████| 576/576 [11:31<00:00,  1.20s/it]


In [74]:
lo_consegui.to_csv('datosBAC.csv')